In [1]:
import pandas as pd

In [30]:
TEST_SPLIT = .2
RANDOM_STATE = 47

In [65]:
landmark_ds = pd.read_csv('Landmark Dataset.csv')
print(f'Positive class: {landmark_ds.IS_Y.sum() / len(landmark_ds.index) * 100:.2f}%')

Positive class: 27.71%


In [66]:
num_of_rows = len(landmark_ds.index)

num_of_test = round(num_of_rows * TEST_SPLIT)
num_of_test = num_of_test - num_of_test % 2  # always even

num_of_train = num_of_rows - num_of_test

# output
num_of_train, num_of_test, num_of_rows

(5457, 1364, 6821)

In [67]:
# Shuffle
landmark_ds = landmark_ds.sample(frac=1., random_state=RANDOM_STATE).reset_index(drop=True)

test_pos = landmark_ds[landmark_ds.IS_Y].sample(num_of_test // 2, random_state=RANDOM_STATE)
test_neg = landmark_ds[~landmark_ds.IS_Y].sample(num_of_test // 2, random_state=RANDOM_STATE)
print(test_pos.index.sort_values())

train_pos = landmark_ds[~landmark_ds.index.isin(test_pos.index) & landmark_ds.IS_Y]
train_neg = landmark_ds[~landmark_ds.index.isin(test_neg.index) & ~landmark_ds.IS_Y]

Int64Index([   5,   15,   26,   44,   52,   53,   66,   75,   84,   90,
            ...
            6719, 6725, 6730, 6756, 6764, 6777, 6793, 6796, 6804, 6813],
           dtype='int64', length=682)


In [68]:
assert not train_pos.index.isin(test_pos.index).any()
assert not train_neg.index.isin(test_neg.index).any()
assert len(train_pos.index) + len(train_neg.index) + len(test_pos) + len(test_neg) == num_of_rows

In [74]:
train_df = pd.concat((train_pos, train_neg)).sample(frac=1., random_state=RANDOM_STATE).reset_index(drop=True)
test_df = pd.concat((test_pos, test_neg)).sample(frac=1., random_state=RANDOM_STATE).reset_index(drop=True)

print(f'Positive train: {train_df.IS_Y.sum() / len(train_df.index) * 100:.2f}%')
print(f'Positive test: {test_df.IS_Y.sum() / len(test_df.index) * 100:.2f}%')

Positive train: 22.14%
Positive test: 50.00%


In [75]:
assert len(train_df.index) == num_of_train
assert len(test_df.index) == num_of_test

In [76]:
train_df.to_csv('landmark_train.csv', index=None)
test_df.to_csv('landmark_test.csv', index=None)

In [2]:
# landmark_ds = pd.read_csv('Landmark Dataset.csv')
# landmark_ds = landmark_ds.sample(frac=1, random_state=47)

# ds_groupped = landmark_ds[landmark_ds.IS_Y], landmark_ds[~landmark_ds.IS_Y]

# train_group = []
# val_group = []
# test_group = []

# for ds in ds_groupped:
#     ds = ds.reset_index(drop=True)
#     ds_len = len(ds.index)
#     train_start = round(.6*ds_len)
#     val_start = train_start + round(.2*ds_len)

#     train_ds = ds.loc[:train_start]
#     val_ds = ds.loc[train_start+1:val_start]
#     test_ds = ds.loc[val_start+1:]
    
#     train_group.append(train_ds)
#     val_group.append(val_ds)
#     test_group.append(test_ds)
#     assert (len(train_ds.index) + len(val_ds.index) + len(test_ds.index)) == ds_len, 'Length not equal'


In [3]:
# train_ds = pd.concat(train_group).reset_index(drop=True)
# val_ds = pd.concat(val_group).reset_index(drop=True)
# test_ds = pd.concat(test_group).reset_index(drop=True)

In [4]:
# train_ds.to_csv('landmark_train.csv', index=None)
# val_ds.to_csv('landmark_val.csv', index=None)
# test_ds.to_csv('landmark_test.csv', index=None)